<h1 align="center" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Iris Flowers
</font>
</h1>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
In this exercise, we aim to implement an algorithm called "k-Nearest Neighbors" or <a href="https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm" target="_blank" style="color:#0099cc">k-NN</a>, which is one of the common machine learning algorithms. By completing this exercise, we will review topics such as vectorized computations using array broadcasting, the use of NumPy's aggregate and universal functions, and indexing with arrays.
</font>
</p>

In [ ]:
import numpy as np

<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Dataset
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
In this exercise, you have been provided with a well-known dataset called <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set" target="_blank" style="color:#0099cc">Iris Flower</a>, which contains the characteristics of 150 iris flowers. Each of these flowers belongs to one of three types: Iris setosa, Iris versicolor, or Iris virginica. The characteristics of each flower include the length and width of the sepal and petal. Using these characteristics, we can imagine each flower as a point in a four-dimensional space, where these four quantities determine the coordinates of that point.
</font>
</p>

<center>
<div style="max-width: 900px">
<img src="img/iris_types.jpg">
</div>
</center>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
The characteristics of 120 iris flowers are stored in a file named <code>irises.npy</code>. When this file is read, it creates a 120 by 4 array, as shown below, where each row represents the characteristics of one flower (sepal length, sepal width, petal length, and petal width in centimeters).
</font>
</p>

<center>
<div style="max-width: 700px">
<img src="img/fig_irises.png">
</div>
</center>


In [ ]:
irises = np.load('irises.npy')
print(irises.shape)


<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
On the other hand, the types of these flowers (setosa, versicolor, or virginica) are stored numerically in another file named <code>types.npy</code>. The values in this array are <code>0</code>, <code>1</code>, or <code>2</code>, which respectively represent the Iris setosa, Iris versicolor, and Iris virginica types.
</font>
</p>

In [ ]:
types = np.load('data/types.npy')
print(types.shape)

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
In this exercise, we aim to implement an algorithm that predicts the type of an iris flower based on its characteristics. This prediction will be made using samples whose types we already know, i.e., the data you have read so far. From now on, we will refer to such samples as <b style="color:#0099cc">training samples</b>.
<br>
The characteristics of 30 other iris flowers, whose types we <b>do not know</b> and which we will predict using the k-NN algorithm, are provided in another file named <code>new_irises.npz</code>. When this file is read, it creates a 30 by 4 array, as shown below, where each row represents the characteristics of one flower. We will refer to such samples as <b style="color:#0099cc">test samples</b>.
</font>
</p>

<center>
<div style="max-width: 700px">
<img src="img/fig_new_irises.png">
</div>
</center>

In [ ]:
new_irises = np.load('data/new_irises.npy')
print(new_irises.shape)

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
We have successfully read all the dataset files. Now, let us denote the number of training samples as <code>n</code> and the number of test samples as <code>m</code>, and store these values in variables with the same names.
</font>
</p>

In [ ]:
n, m = len(irises), len(new_irises)
print("Number of training samples (n):", n)
print("Number of test samples (m):", m)

<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
k-Nearest Neighbors Algorithm
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Suppose you are given the characteristics of an iris flower and need to determine its type. The first idea that likely comes to mind is to measure its similarity to flowers whose types are known and predict its type based on this similarity. Imagine you have found the 3 flowers most similar to this one. You know that, for example, 2 of these flowers are of the virginica type, and 1 is of the setosa type. You would likely agree that, based on majority voting, it is more probable that our flower is also of the virginica type. Thus, we declare this type as our prediction.
<br>
This is exactly what the k-NN algorithm does. However, in this algorithm, instead of finding the greatest similarity, the smallest distance (difference) is typically calculated. The steps of this algorithm are as follows:

<ol>
<li>First, we calculate the distance between the test sample (the sample whose type we want to predict) and all training samples (the samples whose types we know).</li>
<li>Next, we find the <code>k</code> training samples that have the smallest distance to our test sample.</li>
<li>Then, we determine which type the majority of these <code>k</code> samples belong to. We declare that type as our prediction for the test sample.</li>
</ol>
</font>
</p>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Now, let's begin and implement each step of this algorithm using NumPy.
</font>
</p>




<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Part 1: Calculating Distances
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Initially, we aim to write a function that calculates the distance between each sample/row of one array, such as <code>new_points</code>, and each sample/row of another array, such as <code>points</code>. For example, in this exercise, if we have 120 training samples and 30 test samples, we need to calculate the distance between each test sample and each training sample, resulting in an output with dimensions 30 by 120. For instance, the first row of this matrix, as shown on the right in the figure below, contains the distances between the first test sample and each of the training samples.
</font>
</p>

<center>
<div style="max-width: 1000px">
<img src="img/fig.png">
</div>
</center>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
To calculate the distance between two points <code>p</code> and <code>q</code> with <code>f</code> features, we will use the following formula:
</font>
</p>

$$ d(p,q) = {\sum_{i=0}^{f-1} (p_i - q_i)^2} $$

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
In our exercise, <code>f</code> is equal to <code>4</code>. This means that to calculate the distance between two samples, we subtract their sepal widths, square the result, and then do the same for sepal length, petal width, and petal length, summing the results for all four features. There are various methods to calculate the distance between elements of two arrays, and we will experiment with three different approaches.
</font>
</p>

<h3 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Two-Loop Method
</font>
</h3>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
First, we want to perform this task simply using two loops (<code>for</code>). In this method, we write one loop over the test samples and, inside it, another loop over the training samples to calculate the distance between each test sample and each training sample.
<br>
In the function below, you need to replace the <code>None</code> with an expression that calculates the distance between the two points <code>new_points[i]</code> and <code>points[j]</code> using the formula provided above.
<br>
<b style="color:green">Hint:</b>
We recommend using only NumPy's square function (<code>np.square</code>) and sum function (<code>np.sum</code>) in this part.
</font>
</p>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
<b style="color:green">Note:</b> <b>I have no idea how to solve the distance calculation part! ☹</b><br>
Let’s proceed with an example. Our function takes two input matrices named <code>points</code> and <code>new_points</code>. Suppose the first row of each matrix is as follows:<br>
<code>new_points[0]</code><br>
<code>array([5. , 3.6, 1.4, 0.2])</code><br>
<code>points[0]</code><br>
<code>array([5.1, 3.5, 1.4, 0.2])</code><br>
First, we calculate the distance between these two vectors (element-wise subtraction):<br>
<code>new_points[0] - points[0]</code><br>
<code>array([-0.1, 0.1, 0. , 0. ])</code><br>
Next, we square the resulting vector:<br>
<code>np.square(new_points[0] - points[0])</code><br>
<code>array([0.01, 0.01, 0. , 0. ])</code><br>
Finally, we sum all the values in this vector to obtain a single number representing the sum of squared distances:<br>
<code>np.sum(np.square(new_points[0] - points[0]))</code><br>
<code>0.02</code><br>
Now, you need to write code that performs the same calculation, which we did for just one row of the first matrix and one row of the second matrix, for all rows of both matrices.<br>
To perform these calculations for all rows of the first matrix with each row of the second matrix, three different approaches can be used, as explained in the exercise.
</font>
</p>

In [ ]:
def calc_two_loops(new_points, points):
    
    #‌ m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # For each point
        for j in range(n):
            # Calculate the distance between the two points
            d[i, j] = None # TO-DO
            
    return d

In [ ]:
d2 = calc_two_loops(new_irises, irises)
print(d2.shape)

<h3 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Single-Loop Method
</font>
</h3>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Now, we want to utilize the broadcasting or reshaping properties of arrays to eliminate each loop in turn. In this part, we aim to use array broadcasting to remove the inner loop and calculate the distance between each test sample and all training samples simultaneously. In the function below, replace <code>None</code> with an expression that calculates the distances between all training samples and the test sample <code>new_points[i]</code>.
<br><br>
<b style="color:green">Note:</b>
The output of this part is exactly the same as the output of the two-loop function, with the difference that in the previous function, one element of the matrix was calculated at each step, whereas in this part, an entire row of the matrix is calculated at each step.
</font>
</p>

In [ ]:
def calc_one_loop(new_points, points):
    
    # m is the number of new points (test samples)
    m = len(new_points)
    # n is the number of points (training samples)
    n = len(points)
    # Distance matrix
    d = np.zeros((m, n))
    
    # For each new point
    for i in range(m):
        # Calculate the distance between the new point and all the points
        d[i] = None # TO-DO
        
    return d

In [ ]:
d1 = calc_one_loop(new_irises, irises)
print(d1.shape)

<h3 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
No-Loop Method
</font>
</h3>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
This time, we will again use the properties of arrays and write just one line of code in the function below, in front of <code>return</code>, to output all the distances.
<br>
Creating the distance matrix without loops may seem complex. However, using NumPy functions instead of loops makes the code both faster and more concise, and it’s good to get accustomed to this approach. You can test the speed improvement by providing large arrays as input.
</font>
</p>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
<b style="color:green">Hint 1:</b> One possible solution is to reshape the <code>new_points</code> matrix so that it becomes <code>m</code> arrays with dimensions <code>(1, f)</code> (where <code>f</code> is the number of features, which is <code>4</code> in this exercise) and reshape <code>points</code> into <code>1</code> array with dimensions <code>(n, f)</code>. Then, sequentially calculate the distances, square the distances, and compute the sum. Note that all these operations must be performed in a single line of code.
</font>
</p>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
<b style="color:green">Hint 2:</b> For reshaping, you can use <code>reshape</code>, or you can utilize <code>np.newaxis</code>, which was introduced in the lesson on array indexing.
</font>
</p>

In [ ]:
def calc_no_loop(new_points, points):
    return None # TO-DO

In [ ]:
d = calc_no_loop(new_irises, irises)
print(d.shape)

<h3 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Verification
🧐
</font>
</h3>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Calculating the distance array using the three methods above should not result in any differences in the output. Therefore, run the code below to ensure that the outputs of the three functions are equivalent. From now on, we will use the <code>d</code> array as the distance array.
</font>
</p>

In [ ]:
if np.allclose(d, d1, 1e-5) and np.allclose(d, d2, 1e-5) and np.allclose(d1, d2, 1e-5):
    print('Fine!')
else:
    print('There is something wrong!')

<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Part 2: Finding the <code>k</code> Nearest Neighbors
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Using the implemented functions, we were able to calculate the distance between each test sample and each training sample, storing the results in <code>d</code>. Specifically, the distance between the test sample <code>new_irises[i]</code> and the training sample <code>irises[j]</code> is stored in the element <code>d[i][j]</code>. Now, for each test sample, we need to find the <code>k</code> training samples with the smallest distances to it. Note that you should store the indices of the nearest samples, not the distance values themselves. Here, consider <code>k</code> to be <code>10</code>, so the output array <code>k_nearest</code> will be a matrix with dimensions <code>(30, 10)</code>.

<br><br>
<b style="color:green">Hint:</b>
To find the nearest neighbors (smallest distances), use <code>np.argpartition</code>. The difference between <code>argpartition</code> and <code>partition</code> is similar to the difference between <code>argsort</code> and <code>sort</code>. That is, this function outputs the indices of the sorted elements, not their values.
</font>
</p>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
<b style="color:green">Note: </b><b>Why not use <code>argsort</code> instead of <code>argpartition</code>?</b><br>
To find the nearest neighbors (smallest distances), you can use both <code>argsort</code> and <code>argpartition</code>, and the final result of both methods will be correct. However, note that using <code>argsort</code> results in a complete sort, whereas we only need the first <code>k</code> elements. Consequently, using <code>argpartition</code> (especially when dealing with large arrays) will be less computationally expensive.
</font>
</p>

In [ ]:
k = 10
k_nearest = None # TO-DO
print(k_nearest)

<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Part 3: Finding the Type of the <code>k</code> Nearest Neighbors
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
We now have the indices of the nearest neighbors (training samples) for each test sample in the <code>k_nearest</code> array. However, we are interested in the type of each of these neighbors. To replace each index with the corresponding flower type, you can use the <code>types</code> array. Perform this operation in a single line and assign the result to the variable <code>k_nearest_types</code>.

<br><br>
<b style="color:green">Note: </b><b>How does the <code>np.argpartition</code> function work?</b><br>
Suppose our array <code>arr</code> is as follows:<br>
<table style="width:100%; text-align:center">
  <tr><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td></tr>
  <tr><td>80</td><td>50</td><td>20</td><td>30</td><td>-12</td><td>60</td><td>50</td><td>40</td></tr>
</table><br>
If we write <code>np.partition(arr, 3)</code> or <code>np.argpartition(arr, 3)</code>, it means that the 3 elements with the smallest values in the entire array will be brought to the beginning of the array. Thus, we will have:<br>
<table style="width:100%; text-align:center">
  <tr><td>4</td><td>3</td><td>2</td><td>7</td><td>1</td><td>5</td><td>6</td><td>0</td></tr>
  <tr><td>-12</td><td>30</td><td>20</td><td>40</td><td>50</td><td>60</td><td>50</td><td>80</td></tr>
</table><br>
Now, if we used <code>np.partition</code>, the values themselves (bottom row of the table) are returned, but if we used <code>np.argpartition</code>, the indices of the elements (top row of the table) are returned.<br>
Note that if your array is multidimensional, you need to specify the <code>axis</code> argument as well.

<br><br>
<b style="color:green">Note: </b><b>How do I find the type of the nearest neighbors in Part 3?</b><br>
The type of each training sample is stored in the <code>types</code> array, and you can use the index of a sample to retrieve its type from this array. For example, if you want the types of flowers 0 and 50, you can write <code>types[[0, 50]]</code>, which will return their types, yielding the output <code>[0, 1]</code>.
</font>
</p>

In [ ]:
k_nearest_types = None # TO-DO
k_nearest_types

<h2 align="left" style="line-height:200%;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Part 4: Determining the Type of New Flowers
</font>
</h2>

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
Finally, we need to create an array named <code>predicted_types</code> with length <code>m</code>, where the element <code>predicted_types[i]</code> contains the most frequent type identifier in <code>k_nearest_types[i]</code> as the predicted type identifier for <code>new_irises[i]</code>.
<br>
Since NumPy does not directly provide a <code>mode</code> function, we have performed this task using the <code>mode</code> function from the <code>scipy</code> library.
</font>
</p>

In [ ]:
from scipy import stats
predicted_types = stats.mode(k_nearest_types, axis=1).mode.reshape(m)
predicted_types

<p style="direction: ltr; text-align: justify; line-height: 200%; font-family: Arial; font-size: medium">
<font face="Arial" size=3>
In this part, the array <code>new_types</code> has been loaded, containing the correct answers. Using this, calculate the classification accuracy and store it in the variable <code>accuracy</code>. The classification accuracy is the number of correct predictions divided by the number of test samples. If you have followed the steps correctly, the accuracy should be 1 (meaning the classifier correctly predicted the type of all test flowers).
</font>
</p>

In [ ]:
new_types = np.load('data/new_types.npy')
accuracy = None # TO-DO
print('Accuracy:', accuracy)